In [40]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [41]:
!gdown --id 1f_c3uKTDQ4DR3CrwMSI8qdsTKJvKVt7p

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1f_c3uKTDQ4DR3CrwMSI8qdsTKJvKVt7p
To: /kaggle/working/hrnet_w48_coco_wholebody_384x288-6e061c6a_20200922.pth
100%|█████████████████████████████████████████| 255M/255M [00:01<00:00, 245MB/s]


In [42]:
import os
import regex as re
import pandas as pd
path='/kaggle/input/adjective-trainer'
dic={}
lf=[]
df = pd.DataFrame(columns=['File','desc'])
for (root,dirs,files) in os.walk(path,topdown=True):
    
    l=[([i for i in files],root)]
    lf.append(l)

In [43]:
l=[]
for i in range(len(lf)) :
    
     for j in lf[i][0][0]:
         l.append([j,lf[i][0][1]])

In [44]:
import pandas as pd
df=pd.DataFrame(l,columns=['file','root'])
df.shape

(791, 2)

In [45]:
df.head()

,file,root
0,MVI_5295.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...
1,MVI_5138.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...
2,MVI_5297.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...
3,MVI_5136.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...
4,MVI_9408.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...


In [46]:
df['full']=df['root']+'/'+df['file']
x='extra'
dfn=[]
for i in df.full:
    if x not in i.lower().split('/'):
        dfn.append(i)


In [47]:
len(df)

791

In [48]:
dfn[0]

'/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV'

In [49]:
import sys
import argparse
import os
import pprint

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as f
# import torch.optim
# import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms

sys.path.append('/kaggle/input/wholepose/wholepose')
from pose_hrnet import get_pose_net
# import coremltools as ct
from collections import OrderedDict
from config import cfg
from config import update_config

from PIL import Image
import numpy as np
import cv2
from utils import pose_process, plot_pose


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

index_mirror = np.concatenate([
                [1,3,2,5,4,7,6,9,8,11,10,13,12,15,14,17,16],
                [21,22,23,18,19,20],
                np.arange(40,23,-1), np.arange(50,40,-1),
                np.arange(51,55), np.arange(59,54,-1),
                [69,68,67,66,71,70], [63,62,61,60,65,64],
                np.arange(78,71,-1), np.arange(83,78,-1),
                [88,87,86,85,84,91,90,89],
                np.arange(113,134), np.arange(92,113)
                ]) - 1
assert(index_mirror.shape[0] == 133)

multi_scales = [512,640]
def norm_numpy_totensor(img):
    img = img.astype(np.float32) / 255.0
    for i in range(3):
        img[:, :, :, i] = (img[:, :, :, i] - mean[i]) / std[i]
    return torch.from_numpy(img).permute(0, 3, 1, 2)
def stack_flip(img):
    img_flip = cv2.flip(img, 1)
    img_flip = torch.Tensor(img_flip)
    return np.stack([img, img_flip], axis=0)

def merge_hm(hms_list):
    assert isinstance(hms_list, list)
    for hms in hms_list:
        hms[1,:,:,:] = torch.flip(hms[1,index_mirror,:,:], [2])
    
    hm = torch.cat(hms_list, dim=0)
    # print(hm.size(0))
    hm = torch.mean(hms, dim=0)
    return hm

    

In [51]:
with torch.no_grad():
        config = '/kaggle/input/wholepose/wholepose/wholebody_w48_384x288.yaml'
        cfg.merge_from_file(config)

        # dump_input = torch.randn(1, 3, 256, 256)
        # newmodel = PoseHighResolutionNet()
        newmodel = get_pose_net(cfg, is_train=False)
        #print(newmodel)
        # dump_output = newmodel(dump_input)
        # print(dump_output.size())
        checkpoint = torch.load('/kaggle/working/hrnet_w48_coco_wholebody_384x288-6e061c6a_20200922.pth')
        # newmodel.load_state_dict(checkpoint['state_dict'])


        state_dict = checkpoint['state_dict']
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if 'backbone.' in k:
                name = k[9:] # remove module.
            if 'keypoint_head.' in k:
                name = k[14:] # remove module.
            new_state_dict[name] = v
        newmodel.load_state_dict(new_state_dict)

        newmodel.cuda().eval()

        transform  = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

      
        
for i in dfn:
     cap = cv2.VideoCapture(i)
     frame_width = int(cap.get(3))
     frame_height = int(cap.get(4))
     output_list = []
     while cap.isOpened():
            success, img = cap.read()
            if not success:
                    print("Ignoring empty camera frame.")
                    # If loading a video, use 'break' instead of 'continue'.
                    break
            img = cv2.resize(img, (256,256))
            frame_height, frame_width = img.shape[:2]
            img = cv2.flip(img, flipCode=1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            out = []
            for scale in multi_scales:
                    if scale != 512:
                        img_temp = cv2.resize(img, (scale,scale))
                    else:
                        img_temp = img
                    img_temp = stack_flip(img_temp)
                    img_temp = norm_numpy_totensor(img_temp).cuda()
                    hms = newmodel(img_temp)
                    print(type(hms))
                    if scale != 512:
                        out.append(np.array(f.interpolate(hms, (frame_width // 4,frame_height // 4), mode='bilinear').tolist()))
                    else:
                        out.append(np.array(hms.tolist()))
                    out = merge_hm(torch.from_numpy(out))
                    result = out.reshape((133,-1))
                    result = torch.argmax(result, dim=1)
                    result = result.cpu().numpy().squeeze()
                    y = result // (frame_width // 4)
                    x = result % (frame_width // 4)
                    pred = np.zeros((133, 3), dtype=np.float32)
                    pred[:, 0] = x
                    pred[:, 1] = y
                    hm = out.cpu().detach().numpy().reshape((133, frame_height//4, frame_height//4))
                    pred = pose_process(pred, hm)
                    pred[:,:2] *= 4.0 
                    print(pred.shape)
                    assert pred.shape == (133, 3)
                    output_list.append(pred)
            output_list = np.array(output_list)
            np.save(output_npy, output_list)
            cap.release()
                
                    
                    
                    
                    
            
            
            
            
            
    
    
    

<class 'torch.Tensor'>


TypeError: expected np.ndarray (got list)